Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(*)
FROM stackoverflow.posts
WHERE (score > 300 OR favorites_count >= 100) AND post_type_id = (SELECT id FROM stackoverflow.post_types WHERE type = 'Question');


Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH questions AS (
  SELECT DATE_TRUNC('day', creation_date)::date AS question_date,
         COUNT(*) AS count
  FROM stackoverflow.posts
  JOIN stackoverflow.post_types ON stackoverflow.posts.post_type_id = stackoverflow.post_types.id
  WHERE stackoverflow.post_types.type = 'Question'
    AND creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'
  GROUP BY question_date
)
SELECT ROUND(AVG(count))::int AS average_questions_per_day
FROM questions;

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT stackoverflow.users.id) AS users_with_badges_on_registration_day
FROM stackoverflow.users
JOIN stackoverflow.badges
ON stackoverflow.users.id = stackoverflow.badges.user_id
WHERE DATE_TRUNC('day', stackoverflow.users.creation_date) = DATE_TRUNC('day', stackoverflow.badges.creation_date);

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT posts.id) AS unique_posts_with_votes
FROM stackoverflow.users
JOIN stackoverflow.posts ON users.id = posts.user_id
LEFT JOIN stackoverflow.votes ON posts.id = votes.post_id
WHERE users.display_name = 'Joel Coehoorn' AND votes.id IS NOT NULL;


Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       RANK() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT votes.user_id AS user_votes,
       COUNT (votes.id) AS votes_cnt
FROM stackoverflow.votes
JOIN stackoverflow.vote_types
ON vote_types.id = stackoverflow.votes.vote_type_id
WHERE vote_types.name = 'Close'  
GROUP BY user_votes
ORDER BY votes_cnt DESC,
         user_votes DESC
LIMIT 10;


Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(id),
       DENSE_RANK() OVER (ORDER BY COUNT(id) DESC)
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY COUNT(id) DESC,
         user_id
LIMIT 10;


Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title, user_id, score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))::int
FROM stackoverflow.posts     
WHERE title IS NOT NULL
      AND score != 0; 

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE title IS NOT NULL AND
      user_id IN (
              SELECT user_id 
              FROM stackoverflow.badges
              GROUP BY user_id
              HAVING COUNT(id) > 1000);

Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE
          WHEN views >= 350 THEN 1
          WHEN views >= 100 THEN 2
          ELSE 3
       END AS group
FROM stackoverflow.users
WHERE TRIM(location) LIKE '%Canada%'
      AND views > 0
ORDER BY views DESC;

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH canadian_users AS
(SELECT id AS user_id,
        views AS views_cnt,
        CASE
           WHEN views < 100 THEN 3
           WHEN views >= 100 AND views < 350  THEN 2
           ELSE 1
        END AS groups
FROM stackoverflow.users
WHERE TRIM(location) LIKE '%Canada%' AND views > 0
)

SELECT user_id,
       groups,
       views_cnt
FROM (   
          SELECT user_id,
                 views_cnt,
                 groups,
                 MAX(views_cnt) OVER (PARTITION BY groups ORDER BY views_cnt DESC) AS max_views
            FROM canadian_users
         ) AS max_canadian
WHERE views_cnt =  max_views
ORDER BY views_cnt DESC, user_id;


Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [ ]:
SELECT RANK() OVER (ORDER BY days) AS rank,
       users_cnt,
       SUM(users_cnt) OVER (ORDER BY days)::int AS cum
FROM (
    SELECT CAST(DATE_TRUNC('day', creation_date) AS date) AS days,
           COUNT(id) AS users_cnt
    FROM stackoverflow.users
    WHERE DATE_TRUNC('day', creation_date) BETWEEN '2008-11-01' AND '2008-11-30'
    GROUP BY CAST(DATE_TRUNC('day', creation_date) AS date)
) AS t1;

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [ ]:
WITH first_posts AS 
(
    SELECT user_id, 
           creation_date,
           RANK() OVER (PARTITION BY user_id ORDER BY creation_date ASC) AS first_pub_rank
    FROM stackoverflow.posts
)

SELECT first_posts.user_id,
       first_posts.creation_date - stackoverflow.users.creation_date AS days_between_first_post_and_registration
FROM first_posts
JOIN stackoverflow.users ON first_posts.user_id = stackoverflow.users.id
WHERE first_posts.first_pub_rank = 1
ORDER BY first_posts.user_id;

Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date AS month_date,
       SUM(views_count) AS total_views
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date) = 2008
GROUP BY DATE_TRUNC('month', creation_date)
ORDER BY total_views DESC;

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT display_name,
       COUNT(DISTINCT user_id)
FROM stackoverflow.posts
JOIN stackoverflow.users ON posts.user_id=users.id
JOIN stackoverflow.post_types ON post_types.id=posts.post_type_id
WHERE posts.creation_date::date BETWEEN users.creation_date::date AND (users.creation_date::date + INTERVAL '1 month') 
      AND post_types.type LIKE 'Answer'
GROUP BY display_name
HAVING COUNT(posts.id) > 100
ORDER BY display_name;

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH users AS
(
SELECT users.id
FROM stackoverflow.posts
JOIN stackoverflow.users ON posts.user_id=users.id
WHERE (users.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30')
AND (posts.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31')
GROUP BY users.id
)

SELECT DATE_TRUNC('month', posts.creation_date)::date AS month,
       COUNT(posts.id)
FROM stackoverflow.posts
WHERE posts.user_id IN 
(
SELECT id
FROM users
)
AND DATE_TRUNC('year', posts.creation_date)::date = '2008-01-01'
GROUP BY DATE_TRUNC('month', posts.creation_date)::date
ORDER BY DATE_TRUNC('month', posts.creation_date)::date DESC;

Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумма просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id, creation_date, views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date) AS cumulative_count
FROM stackoverflow.posts
ORDER BY user_id, creation_date;

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH temp AS
(
SELECT user_id,
       COUNT(DISTINCT DATE_TRUNC('day', creation_date)::date) AS days_active
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY user_id
)
SELECT ROUND(AVG(days_active))::int AS average_days_active
FROM temp;

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
Номер месяца.
Количество постов за месяц.
Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
WITH temp AS
(
SELECT EXTRACT(MONTH FROM creation_date)::int AS month,
       COUNT(id) AS posts_count
FROM stackoverflow.posts
WHERE creation_date >= '2008-09-01' AND creation_date <= '2008-12-31'
GROUP BY month
)
SELECT month,
       posts_count,
       ROUND((posts_count::numeric / LAG(posts_count) OVER (ORDER BY month) - 1) * 100, 2) AS change_percentage
FROM temp
ORDER BY month;

Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH most_active_user AS (
    SELECT user_id
    FROM stackoverflow.posts
    GROUP BY user_id
    ORDER BY COUNT(id) DESC
    LIMIT 1
)

, october_posts AS (
    SELECT
        EXTRACT(WEEK FROM creation_date) AS week_number,
        creation_date,
        ROW_NUMBER() OVER (PARTITION BY EXTRACT(WEEK FROM creation_date) ORDER BY creation_date DESC) AS rn
    FROM stackoverflow.posts
    WHERE user_id = (SELECT user_id FROM most_active_user)
          AND EXTRACT(MONTH FROM creation_date) = 10
          AND EXTRACT(YEAR FROM creation_date) = 2008
)

SELECT week_number, MAX(creation_date) AS last_post_date
FROM october_posts
WHERE rn = 1
GROUP BY week_number
ORDER BY week_number;